In [34]:
import numpy as np
import scipy
from matplotlib import pyplot
from scipy import integrate

In [40]:
alpha = 0.05

Ig = np.array([83, 85,
               84, 85, 85, 86, 86, 87,
               86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90,
               89, 90, 90, 91,
               90, 92])
Group = [1, 1,
         2, 2, 2, 2, 2, 2, 
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
         4, 4, 4, 4,
         5, 5]

Psi = np.zeros(len(Group) * 5).reshape(len(Group), 5)
for i in range(len(Group)):
    Psi[i][Group[i]-1] = 1 

# a) Влияние возраста

## Уравнение регрессии

In [70]:
def linear_regression(eta, Psi, n, p):
    F = np.dot(Psi.T, Psi)
    F_inv = np.linalg.inv(F)
    beta = np.dot(np.dot(F_inv, Psi.T), eta)
    
    e = eta - np.dot(Psi, beta.T)
    RSS = np.dot(e.T, e)
    mean = np.mean(eta)
    TSS = np.sum([(etai - mean)**2 for etai in eta])
    R_2 = 1 - RSS/TSS
    delta_r = R_2 * TSS / RSS * (n-p) / (p-1)
    
    delta = []
    for i in range(0, 5):
        delta.append(beta[i]*((n-p)**0.5)/(RSS * F_inv[i, i])**0.5)

    return beta, delta_r, delta, R_2, F_inv, RSS

In [77]:
def possibility(delta, N, p, counter):
    n = N - p
    I = integrate.quad(lambda x: scipy.special.gamma(n/2 + 1/2) / (n*np.pi)**0.5 / scipy.special.gamma(n/2) / (1 + x*x/n)**(n/2+1/2), delta, np.inf)
    p_val = 2*I[0]
    print("p-value =", p_val)
    if p_val > alpha:
        print("β", counter, "не значима")
    else:
        print("β", counter, "значима")
    print()

In [78]:
beta, delta_R, delta, R_2, F_inv, RSS = linear_regression(Ig, Psi, len(Ig), 5)
print("Было: η =", np.round(beta, 1), "*", "(I II III IV V)")

Было: η = [84.  85.5 87.8 90.  91. ] * (I II III IV V)


## Значимость коэффициентов

In [79]:
possibility(np.abs(delta[0]), len(Ig), 5, 1)
possibility(np.abs(delta[1]), len(Ig), 5, 2)
possibility(np.abs(delta[2]), len(Ig), 5, 3)
possibility(np.abs(delta[3]), len(Ig), 5, 4)
possibility(np.abs(delta[4]), len(Ig), 5, 5)

p-value = 2.4341091408717638e-29
β 1 значима

p-value = 2.921887806517796e-34
β 2 значима

p-value = 3.992126809615538e-37
β 3 значима

p-value = 6.033643921169127e-33
β 4 значима

p-value = 4.922009905380071e-30
β 5 значима



## Значимость регрессии

In [80]:
print("R^2 =", R_2)
d1 = 5-1
d2 = len(Ig)-5
p_value = integrate.quad(lambda x: ((d1*x)**d1 * d2**d2 / (d1*x + d2)**(d1+d2))**0.5 / (x * scipy.special.beta(d1/2, d2/2)), delta_R, np.inf)[0]
print("p-value =", p_value)
if p_value > alpha/2 and p_value < 1-alpha/2:
    print("Регрессия не значима")
else:
    print("Регрессия значима")

R^2 = 0.8106060606060606
p-value = 5.407410946440219e-07
Регрессия значима


# b) Попарное сравнение средних в рамках регрессионной модели

In [89]:
def possibility_pair(delta, N, p, c1, c2):
    n = N - p
    I = integrate.quad(lambda x: scipy.special.gamma(n/2 + 1/2) / (n*np.pi)**0.5 / scipy.special.gamma(n/2) / (1 + x*x/n)**(n/2+1/2), delta, np.inf)
    p_val = 2*I[0]
    print("p-value =", round(p_val, 8))
    if p_val > alpha:
        print("β", c1, " и ", "β", c2, " одинаковы")
    else:
        print("β", c1, " и ", "β", c2, " не одинаковы")
    print()
    

In [90]:
for i in range(0, 5):
    for j in range(i+1, 5):
        delta_ij = np.abs((beta[i] - beta[j])) / (RSS * (F_inv[i,i] + F_inv[j,j]))**0.5 *(len(Ig) - 5)**0.5
        possibility_pair(delta_ij, len(Ig), 5, i, j)

p-value = 0.10310041
β 0  и  β 1  одинаковы

p-value = 0.0001662
β 0  и  β 2  не одинаковы

p-value = 2.78e-06
β 0  и  β 3  не одинаковы

p-value = 2.41e-06
β 0  и  β 4  не одинаковы

p-value = 0.00039504
β 1  и  β 2  не одинаковы

p-value = 2.55e-06
β 1  и  β 3  не одинаковы

p-value = 4.09e-06
β 1  и  β 4  не одинаковы

p-value = 0.00239334
β 2  и  β 3  не одинаковы

p-value = 0.00100256
β 2  и  β 4  не одинаковы

p-value = 0.29579135
β 3  и  β 4  одинаковы

